You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [3]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [4]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [6]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [7]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [8]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [9]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [10]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [11]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)


## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [12]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:

Content Plan: 
Title: The Latest Trends in Artificial Intelligence: A Comprehensive Guide

Outline:
I. Introduction
- Definition of Artificial Intelligence
- Importance of staying updated on AI trends
II. Latest Trends in Artificial Intelligence
- Machine learning advancements
- Natural language processing developments
- AI in healthcare
- Ethical considerations in AI
III. Key Players in Artificial Intelligence
- Google
- IBM
- Microsoft
- Tesla
IV. Noteworthy News on Artificial Intelligence
- Recent AI break

I now can give a great answer

Final Answer: 

# The Latest Trends in Artificial Intelligence: A Comprehensive Guide

## Introduction

Artificial Intelligence (AI) is a rapidly evolving field that has the potential to revolutionize various industries. It refers to the simulation of human intelligence processes by machines, including learning, reasoning, and self-correction. Staying updated on AI trends is crucial for individuals and organizations looking to leverage the latest advancements in technology to enhance their operations and stay competitive in the market.

## Latest Trends in Artificial Intelligence

### Machine Learning Advancements
One of the most significant trends in AI is the continuous advancements in machine learning algorithms. These algorithms enable machines to learn from data, identify patterns, and make decisions with minimal human intervention. Companies are increasingly using machine learning to improve customer experiences, optimize operations, and drive busin

- Display the results of your execution as markdown in the notebook.

In [13]:
from IPython.display import Markdown
Markdown(result)

# The Latest Trends in Artificial Intelligence: A Comprehensive Guide

## Introduction

Artificial Intelligence (AI) is a rapidly evolving field that has the potential to revolutionize various industries. It refers to the simulation of human intelligence processes by machines, including learning, reasoning, and self-correction. Staying updated on AI trends is crucial for individuals and organizations looking to leverage the latest advancements in technology to enhance their operations and stay competitive in the market.

## Latest Trends in Artificial Intelligence

### Machine Learning Advancements
One of the most significant trends in AI is the continuous advancements in machine learning algorithms. These algorithms enable machines to learn from data, identify patterns, and make decisions with minimal human intervention. Companies are increasingly using machine learning to improve customer experiences, optimize operations, and drive business growth.

### Natural Language Processing Developments
Natural Language Processing (NLP) is another exciting trend in AI, allowing machines to understand and generate human language. NLP technologies power virtual assistants, chatbots, and language translation services, enhancing communication and efficiency in various applications.

### AI in Healthcare
AI is making significant strides in the healthcare industry, from diagnosing diseases to personalizing treatment plans. Machine learning algorithms analyze medical data to identify patterns and predict outcomes, leading to more accurate diagnoses and improved patient care.

### Ethical Considerations in AI
As AI technologies become more advanced, ethical considerations surrounding their use become increasingly important. Issues such as data privacy, bias in algorithms, and the impact of automation on jobs require careful consideration to ensure that AI benefits society as a whole.

## Key Players in Artificial Intelligence

### Google
Google is a leading player in AI research and development, leveraging AI technologies across its products and services. The company's deep learning algorithms power search engines, voice assistants, and image recognition systems, showcasing the potential of AI in improving user experiences.

### IBM
IBM is known for its advancements in AI, particularly in the areas of cognitive computing and data analytics. The company's Watson AI platform is used in various industries, from healthcare to finance, to drive innovation and solve complex challenges.

### Microsoft
Microsoft has been investing heavily in AI research and applications, integrating AI capabilities into its cloud services and productivity tools. The company's AI initiatives focus on empowering individuals and organizations to achieve more through intelligent technologies.

### Tesla
Tesla is at the forefront of AI innovations in the automotive industry, developing self-driving technology and AI-driven features for its electric vehicles. The company's Autopilot system demonstrates the potential of AI to enhance safety and efficiency on the roads.

## Noteworthy News on Artificial Intelligence

### Recent AI Breakthroughs
Recent AI breakthroughs include advancements in deep learning, reinforcement learning, and computer vision. These breakthroughs are driving the development of new AI applications and pushing the boundaries of what machines can achieve.

### AI Applications in Various Industries
AI is being applied across various industries, from finance and retail to transportation and agriculture. Companies are using AI to automate processes, gain insights from data, and deliver personalized experiences to customers, leading to increased efficiency and innovation.

### AI Regulations and Policies
Governments and organizations are increasingly focusing on regulating AI technologies to ensure their responsible and ethical use. Policies around data privacy, algorithm transparency, and AI accountability are being developed to address the potential risks associated with AI deployments.

## Target Audience Analysis

### Tech Enthusiasts
Tech enthusiasts are eager to learn about the latest trends and developments in AI, exploring the possibilities of AI technologies and their impact on society.

### Business Professionals
Business professionals are interested in leveraging AI to drive growth and innovation in their organizations, seeking insights on how AI can transform their operations and strategies.

### Students Studying AI or Related Fields
Students studying AI or related fields are keen on staying updated on the latest research and advancements in the field, preparing themselves for future careers in AI.

### Decision-Makers in Companies Exploring AI Solutions
Decision-makers in companies exploring AI solutions are looking for information on the top AI companies, latest AI news, and ethical considerations in AI to make informed decisions about AI implementations.

## SEO Keywords

Artificial Intelligence trends, Latest AI news, Top AI companies, AI developments

## Resources

Industry reports, Research papers, Interviews with AI experts, Case studies on AI implementations

**Call to Action:**
Stay informed on the latest trends in Artificial Intelligence by following reputable sources and engaging with AI professionals in the industry. Keep exploring the possibilities of AI in your field and stay updated on the ethical implications of AI technologies. 

By following the latest trends and developments in Artificial Intelligence, individuals and organizations can harness the power of AI to drive innovation, improve efficiency, and create positive impacts on society. As AI continues to evolve, staying informed and engaged with the industry will be key to unlocking the full potential of this transformative technology.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [14]:
topic = "Use case identification for public sector (e.g, smart contract and procurement monitoring)"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Use case identification for public sector (e.g, smart contract and procurement monitoring).
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Content Plan: Use Case Identification for Public Sector (e.g., Smart Contract and Procurement Monitoring)

1. Introduction
- Brief overview of the importance of use case identification in the public sector
- Definition of smart contracts and procurement monitoring
- Mention of the latest trends in use case identification for public sector

2. Key Points
- Latest Trends:
  - Adoption of blockchain technology for smart contracts in public secto

I now can give a great answer

Final Answer:
# Use Case Identification for Public Sector: Enhancing Efficiency Through Smart Contracts and Procurement Monitoring

## Introduction

In the realm of the public sector, the identification of use cases plays a crucial role in optimizing operations and increasing transparency. Smart contracts, which are self-executing contracts with the terms of the agreement directly written into code, have revolutionized the way government agencies handle transactions. On the other hand, procurement monitoring involves the use of advanced technologies to track and analyze procurement processes for improved efficiency. The latest trends in use case identification for the public sector focus on the adoption of blockchain technology for smart contracts, the implementation of AI-powered procurement monitoring systems, and the integration of IoT devices for real-time data tracking in procurement processes.

## Key Points

### Latest Trends:

- **Adoption of Bloc

In [15]:
Markdown(result)

# Use Case Identification for Public Sector: Enhancing Efficiency Through Smart Contracts and Procurement Monitoring

## Introduction

In the realm of the public sector, the identification of use cases plays a crucial role in optimizing operations and increasing transparency. Smart contracts, which are self-executing contracts with the terms of the agreement directly written into code, have revolutionized the way government agencies handle transactions. On the other hand, procurement monitoring involves the use of advanced technologies to track and analyze procurement processes for improved efficiency. The latest trends in use case identification for the public sector focus on the adoption of blockchain technology for smart contracts, the implementation of AI-powered procurement monitoring systems, and the integration of IoT devices for real-time data tracking in procurement processes.

## Key Points

### Latest Trends:

- **Adoption of Blockchain Technology:** Government agencies are increasingly turning to blockchain technology for implementing smart contracts in public sector projects. This ensures secure and transparent transactions while reducing the risk of fraud.

- **AI-Powered Procurement Monitoring:** The use of artificial intelligence in procurement monitoring systems allows for real-time analysis of procurement data, leading to more informed decision-making and enhanced efficiency in the procurement process.

- **Integration of IoT Devices:** By integrating IoT devices for real-time data tracking in procurement processes, government agencies can streamline operations, monitor inventory levels, and optimize supply chain management.

### Key Players:

- **Government Agencies:** Public sector organizations are exploring various use case identification strategies to modernize their operations and improve service delivery to citizens.

- **Tech Companies:** Innovative solutions for smart contract management are being developed by tech companies to meet the growing demand for efficient and secure transaction processing in government projects.

- **Consultancies:** Consultancies are offering their expertise in procurement monitoring and optimization to help government agencies enhance transparency and accountability in their procurement processes.

### Noteworthy News:

- **Success Stories:** Public sector organizations have reported significant improvements in efficiency through the implementation of use case identification strategies, showcasing the tangible benefits of adopting smart contracts and procurement monitoring solutions.

- **Challenges:** Government agencies face challenges in implementing smart contract solutions, such as regulatory hurdles and the need for skilled personnel to manage these advanced technologies effectively.

- **Regulatory Updates:** Regulatory updates have a significant impact on procurement monitoring practices in the public sector, necessitating continuous monitoring and adaptation to ensure compliance with changing regulations.

## Target Audience

This blog post is tailored for government officials and policymakers looking to modernize public sector operations, IT professionals involved in digital transformation initiatives within government agencies, and procurement managers seeking to enhance transparency and accountability in their processes.

## Call to Action

I encourage readers to explore case studies of successful use case identification projects in the public sector, consider the benefits of implementing smart contracts and procurement monitoring solutions in their organizations, and delve into further learning resources such as whitepapers, webinars, and industry reports for a deeper understanding of the topic.

By encompassing the latest trends, key players, and noteworthy news in the field of use case identification for the public sector, this blog post aims to provide valuable insights to its target audience and promote the adoption of innovative technologies for improved efficiency and transparency in government operations.

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).